In [1]:
%cd message-evaluation-framework/analysis_notebook/

/Users/suvadeep.mukherjee/Documents/psychological_interventions_on_cheating_behavior/message-evaluation-framework/analysis_notebook


/opt/anaconda3/envs/cheating_env/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

project_root = Path('../')
data1_path = project_root / "data" / "step1_evaluations.csv"
data2_path = project_root / "data" / "step2_evaluations.csv"
data1_path

PosixPath('../data/step1_evaluations.csv')

In [3]:
# Load the datasets
step1_df = pd.read_csv(data1_path)
step2_df = pd.read_csv(data2_path)

In [4]:
# --------------------------------------------------------------
# Step 1: Calculate average alignment ratings per message
# --------------------------------------------------------------
alignment_ratings = (step1_df.groupby(['true_concept', 'message_text', 'creator_user_id'])
                     ['alignment_rating']
                     .agg(['count', 'mean', 'std'])
                     .reset_index())
alignment_ratings.rename(columns={'mean': 'avg_alignment', 
                                 'count': 'total_evaluators', 
                                 'std': 'std_alignment',
                                 'true_concept': 'concept'}, inplace=True)

# print(f"Calculated alignment ratings for {len(alignment_ratings)} messages across {alignment_ratings['true_concept'].nunique()} concepts")
alignment_ratings


,concept,message_text,creator_user_id,total_evaluators,avg_alignment,std_alignment
0,Autonomy,Are you aware that it is up to you how you tac...,Expert A,4,9.50,0.577350
1,Autonomy,Can you think of a time when you felt really p...,Expert C,4,7.75,2.629956
2,Autonomy,How do you feel when you're doing something th...,Expert E,4,9.25,0.957427
3,Autonomy,What's it about working on a project that real...,Expert D,4,8.50,1.290994
4,Autonomy,"You're in the midst of a tough project, and it...",Expert B,4,9.25,0.957427
...,...,...,...,...,...,...
70,Vicarious experience,"Can you imagine someone like you, investing th...",Expert A,4,7.50,2.081666
71,Vicarious experience,Have you ever noticed how the success of other...,Expert E,4,7.50,2.081666
72,Vicarious experience,What do you think gives someone the courage to...,Expert C,4,8.75,1.892969
73,Vicarious experience,What's helping you tackle this tough task? You...,Expert D,4,9.25,0.957427


In [5]:
# --------------------------------------------------------------
# Step 2: Extract and calculate average preference ratings per message
# --------------------------------------------------------------
preference_ratings_list = []

# For each row in step2_df, extract the individual message ratings
for _, row in step2_df.iterrows():
    concept_name = row['concept_name']
    evaluator_id = row['evaluator_id']
    
    # Go through each message rating
    for i in range(4):
        message_id_col = f'message_ratings[{i}].message_id'
        
        # Check if this column exists and has a value
        if message_id_col in row.index and not pd.isna(row[message_id_col]):
            message_id = row[message_id_col]
            message_text = row[f'message_ratings[{i}].message_text']
            creator_user_id = row[f'message_ratings[{i}].creator_user_id']
            rating = row[f'message_ratings[{i}].rating']
            
            preference_ratings_list.append({
                'concept_name': concept_name,
                'message_id': message_id,
                'message_text': message_text,
                'creator_user_id': creator_user_id,
                'preference_rating': rating,
                'evaluator_id': evaluator_id
            })

preference_ratings_df = pd.DataFrame(preference_ratings_list)

# Calculate average preference ratings per message
preference_ratings = (preference_ratings_df.groupby(['concept_name', 'message_text', 'creator_user_id'])
                     ['preference_rating']
                     .agg(['mean', 'std'])
                     .reset_index())
preference_ratings.rename(columns={'mean': 'avg_preference', 
                                  'std': 'std_preference',
                                  'concept_name': 'concept'}, inplace=True)

# print(f"Calculated preference ratings for {len(preference_ratings)} messages across {preference_ratings['concept_name'].nunique()} concepts")
preference_ratings


,concept,message_text,creator_user_id,avg_preference,std_preference
0,Autonomy,Are you aware that it is up to you how you tac...,Expert A,8.25,0.957427
1,Autonomy,Can you think of a time when you felt really p...,Expert C,8.25,1.258306
2,Autonomy,How do you feel when you're doing something th...,Expert E,8.00,3.366502
3,Autonomy,What's it about working on a project that real...,Expert D,5.00,1.414214
4,Autonomy,"You're in the midst of a tough project, and it...",Expert B,8.00,1.825742
...,...,...,...,...,...
70,Vicarious experience,"Can you imagine someone like you, investing th...",Expert A,8.00,0.816497
71,Vicarious experience,Have you ever noticed how the success of other...,Expert E,6.25,2.500000
72,Vicarious experience,What do you think gives someone the courage to...,Expert C,6.75,1.500000
73,Vicarious experience,What's helping you tackle this tough task? You...,Expert D,5.25,2.500000


In [6]:
# --------------------------------------------------------------
# Step 3: Merge alignment and preference ratings
# --------------------------------------------------------------

# Merge on concept, message_id, and message_text to ensure correct matching
merged_ratings = pd.merge(alignment_ratings, preference_ratings, 
                          how='outer',
                          on=['concept', 'message_text', 'creator_user_id'])

merged_ratings

# Fill NaN values (in case some messages only have one type of rating)
# merged_ratings['avg_alignment'].fillna(0, inplace=True)
# merged_ratings['avg_preference'].fillna(0, inplace=True)

# print(f"Merged ratings contain {len(merged_ratings)} messages across {merged_ratings['concept'].nunique()} concepts")


,concept,message_text,creator_user_id,total_evaluators,avg_alignment,std_alignment,avg_preference,std_preference
0,Autonomy,Are you aware that it is up to you how you tac...,Expert A,4,9.50,0.577350,8.25,0.957427
1,Autonomy,Can you think of a time when you felt really p...,Expert C,4,7.75,2.629956,8.25,1.258306
2,Autonomy,How do you feel when you're doing something th...,Expert E,4,9.25,0.957427,8.00,3.366502
3,Autonomy,What's it about working on a project that real...,Expert D,4,8.50,1.290994,5.00,1.414214
4,Autonomy,"You're in the midst of a tough project, and it...",Expert B,4,9.25,0.957427,8.00,1.825742
...,...,...,...,...,...,...,...,...
70,Vicarious experience,"Can you imagine someone like you, investing th...",Expert A,4,7.50,2.081666,8.00,0.816497
71,Vicarious experience,Have you ever noticed how the success of other...,Expert E,4,7.50,2.081666,6.25,2.500000
72,Vicarious experience,What do you think gives someone the courage to...,Expert C,4,8.75,1.892969,6.75,1.500000
73,Vicarious experience,What's helping you tackle this tough task? You...,Expert D,4,9.25,0.957427,5.25,2.500000


In [7]:
# --------------------------------------------------------------
# Step 4: Calculate combined score and select top 3 messages per concept
# --------------------------------------------------------------
# Define weights for alignment vs preference
alignment_weight = 0.5
preference_weight = 0.5

# Calculate combined score using weights on the original 1-10 scale values
merged_ratings['combined_score'] = (alignment_weight * merged_ratings['avg_alignment'] + 
                                  preference_weight * merged_ratings['avg_preference'])

# Select top 3 messages for each concept
top_messages = []
for concept, group in merged_ratings.groupby('concept'):
    # Sort by combined score in descending order
    sorted_messages = group.sort_values('combined_score', ascending=False)
    
    # Select top 3 (or fewer if less than 3 messages exist)
    # top_3 = sorted_messages.head(3)
    
    for idx, row in sorted_messages.iterrows():
        top_messages.append({
            'concept': concept,
            'message_text': row['message_text'],
            'creator_user_id': row['creator_user_id'],
            'avg_alignment': row['avg_alignment'],
            'avg_preference': row['avg_preference'],
            'combined_score': row['combined_score'],
            'rank': sorted_messages.index.get_loc(idx) + 1
        })

top_messages_df = pd.DataFrame(top_messages)
top_messages_df.sort_values(['concept', 'rank'], inplace=True)

top_messages_df


,concept,message_text,creator_user_id,avg_alignment,avg_preference,combined_score,rank
0,Autonomy,Are you aware that it is up to you how you tac...,Expert A,9.50,8.25,8.875,1
1,Autonomy,How do you feel when you're doing something th...,Expert E,9.25,8.00,8.625,2
2,Autonomy,"You're in the midst of a tough project, and it...",Expert B,9.25,8.00,8.625,3
3,Autonomy,Can you think of a time when you felt really p...,Expert C,7.75,8.25,8.000,4
4,Autonomy,What's it about working on a project that real...,Expert D,8.50,5.00,6.750,5
...,...,...,...,...,...,...,...
70,Vicarious experience,You're facing a challenge that's got you stump...,Expert B,9.00,7.25,8.125,1
71,Vicarious experience,"Can you imagine someone like you, investing th...",Expert A,7.50,8.00,7.750,2
72,Vicarious experience,What do you think gives someone the courage to...,Expert C,8.75,6.75,7.750,3
73,Vicarious experience,What's helping you tackle this tough task? You...,Expert D,9.25,5.25,7.250,4


In [8]:
detailed_output = top_messages_df.sort_values(['concept', 'combined_score'], ascending=[True, False])
detailed_output.to_csv('../data/all_messages_with_ratings.csv', index=False)
print(f"Saved all messages with detailed ratings to 'all_messages_with_ratings.csv'")

Saved all messages with detailed ratings to 'all_messages_with_ratings.csv'


In [9]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import inch
from datetime import datetime

# Load the data
# Replace with the actual path to your data
data_path = '../data/all_messages_with_ratings.csv'
try:
    df = pd.read_csv(data_path)
except FileNotFoundError:
    # If file not found, try using the top_messages_df from your notebook
    print("CSV file not found. Please run this script after generating the ratings.")
    exit(1)

# Filter to keep only top messages and sort by concept and rank
if 'rank' in df.columns:
    top_messages_df = df[df['rank'] <= 3].sort_values(['concept', 'rank'])
else:
    # If there's no rank column, let's create one by sorting
    top_messages = []
    for concept, group in df.groupby('concept'):
        # Sort by combined score in descending order
        sorted_messages = group.sort_values('combined_score', ascending=False)
        # Take top 3
        top_3 = sorted_messages.head(3)
        top_3['rank'] = range(1, len(top_3) + 1)
        top_messages.append(top_3)
    
    top_messages_df = pd.concat(top_messages)
    top_messages_df = top_messages_df.sort_values(['concept', 'rank'])

# Create PDF
pdf_filename = "Best_3_Messages.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=A4, rightMargin=72, leftMargin=72, topMargin=72, bottomMargin=18)

# Create styles
styles = getSampleStyleSheet()
title_style = styles["Title"]
heading1_style = styles["Heading1"]
heading2_style = styles["Heading2"]
normal_style = styles["Normal"]

# Create custom styles
concept_style = ParagraphStyle(
    'ConceptStyle',
    parent=styles['Heading1'],
    fontSize=10,
    spaceAfter=6,
    backColor=colors.lightgrey,
    borderWidth=1,
    borderColor=colors.black,
    borderPadding=5,
)

# Container for content
content = []

# Title
content.append(Paragraph("Top Concept Messages Report", title_style))
content.append(Spacer(1, 0.25*inch))

# Introduction
intro_text = """This report presents the top-rated messages for each concept based on alignment and preference 
ratings from evaluators. Messages are ranked by their combined score, which weights both how well 
the message communicates its intended concept (alignment) and how motivating/effective evaluators 
found the message (preference)."""
content.append(Paragraph(intro_text, normal_style))
content.append(Spacer(1, 0.25*inch))

# Define the order of theories and their concepts
theory_concepts = {
    "Self-Determination Theory": ["Autonomy", "Competence", "Relatedness"],
    "Cognitive Dissonance Theory": ["Self-Concept", "Cognitive Inconsistency", "Dissonance Arousal", "Dissonance Reduction"],
    "Self-Efficacy Theory": ["Performance Accomplishments", "Vicarious Experience", "Verbal Persuasion", "Emotional Arousal"],
    "Social Norms Theory": ["Descriptive Norms", "Injunctive Norms", "Social Sanctions", "Reference Group Identification"]
}

# Create a mapping of lowercase concept names to their original case
concept_case_map = {c.lower(): c for c in top_messages_df['concept'].unique()}

# Get available concepts from data (case-insensitive comparison)
available_concepts = set([c.lower() for c in top_messages_df['concept'].unique()])

# Create a theory header style
theory_header_style = ParagraphStyle(
    'TheoryStyle',
    parent=styles['Heading1'],
    fontSize=16,
    spaceAfter=10,
    backgroundColor=colors.lightblue,
    borderWidth=1,
    borderColor=colors.black,
    borderPadding=8,
)

# For each theory, create sections for its concepts
for theory, concepts in theory_concepts.items():
    # Add theory heading
    # content.append(Paragraph(f"Theory: {theory}", theory_header_style))
    # content.append(Spacer(1, 0.1*inch))
    
    # Process each concept in this theory
    for concept in concepts:
        # Check if this concept exists in our data (case-insensitive comparison)
        if concept.lower() in available_concepts:
            # Get the actual case from the data
            actual_concept = concept_case_map.get(concept.lower(), concept)
            
            # Concept heading
            content.append(Paragraph(f"Concept: {actual_concept} ({theory})", concept_style))
            content.append(Spacer(1, 0.1*inch))
            
            # Get messages for this concept (case-insensitive filter)
            concept_msgs = top_messages_df[top_messages_df['concept'].str.lower() == actual_concept.lower()]
            
            # If no messages found, try with our specified concept name
            if len(concept_msgs) == 0:
                print(f"No messages found for {actual_concept}, trying with {concept}")
                concept_msgs = top_messages_df[top_messages_df['concept'].str.lower() == concept.lower()]
            
            if len(concept_msgs) > 0:
                # Sort by rank to ensure proper order
                concept_msgs = concept_msgs.sort_values('rank')
                
                for _, row in concept_msgs.iterrows():
                    # Message rank
                    content.append(Paragraph(f"Rank {int(row['rank'])}", heading2_style))
                    
                    # Message text
                    message_text = f"<b>Message:</b> {row['message_text']}"
                    content.append(Paragraph(message_text, normal_style))
                    content.append(Spacer(1, 0.1*inch))
                    
                    # Scores table
                    data = [
                        ["Metric", "Score"],
                        ["Alignment Score", f"{row['avg_alignment']:.2f}"],
                        ["Preference Score", f"{row['avg_preference']:.2f}"],
                        ["Combined Score", f"{row['combined_score']:.2f}"]
                    ]
                    
                    t = Table(data, colWidths=[2*inch, 1*inch])
                    t.setStyle(TableStyle([
                        ('BACKGROUND', (0, 0), (1, 0), colors.lightgrey),
                        ('TEXTCOLOR', (0, 0), (1, 0), colors.black),
                        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
                        ('GRID', (0, 0), (-1, -1), 1, colors.black)
                    ]))
                    
                    content.append(t)
                    content.append(Spacer(1, 0.1*inch))
                    
                    # Creator info
                    creator_text = f"<b>Creator:</b> {row['creator_user_id']}"
                    content.append(Paragraph(creator_text, normal_style))
                    content.append(Spacer(1, 0.25*inch))
            else:
                # If no messages for this concept, add a note
                content.append(Paragraph("No messages available for this concept.", normal_style))
                content.append(Spacer(1, 0.25*inch))
            # Add page break after each concept
            content.append(PageBreak())
        else:
            # If concept doesn't exist in data, just add the heading
            content.append(Paragraph(f"Concept: {concept}", concept_style))
            content.append(Spacer(1, 0.1*inch))
            content.append(Paragraph("No messages available for this concept.", normal_style))
            content.append(Spacer(1, 0.25*inch))
            # Add page break after each concept
            content.append(PageBreak())
    
    # No page break after each theory - we'll add them after each concept instead

# Add footer with date
# footer_text = f"Report generated on: {datetime.now().strftime('%Y-%m-%d')}"
# content.append(Paragraph(footer_text, normal_style))

# Build the PDF
doc.build(content)

print(f"Report generated: {pdf_filename}")

Report generated: Best_3_Messages.pdf
